In [1]:
import os
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

In [2]:
import os
import pandas as pd
import numpy as np

# Path to frame directory (update if needed)
frames_dir = "/Users/berkeayyildizli/ENS_Graduation_Project/data/images"
frame_files = sorted([f for f in os.listdir(frames_dir) if f.startswith("Frame_") and f.endswith(".csv")])

# Initialize list to store features
feature_list = []

# Debug: Check how many frames are found
print(f"Total Frames Found: {len(frame_files)}")

for file_name in frame_files:
    file_path = os.path.join(frames_dir, file_name)
    frame_data = pd.read_csv(file_path, header=None)

    # Check for empty frames
    if frame_data.shape[0] == 0 or frame_data.shape[1] == 0:
        print(f"🚨 Skipping {file_name} - Empty frame!")
        continue

    # Convert to numpy array
    frame_np = frame_data.to_numpy(dtype=np.float32)

    # Replace zeros with NaNs
    frame_np[frame_np == 0] = np.nan

    # Mean imputation (column-wise)
    imputer = SimpleImputer(strategy="mean")
    frame_np_imputed = imputer.fit_transform(frame_np)

    # Compute statistical features on imputed data
    min_temp = np.min(frame_np_imputed)
    max_temp = np.max(frame_np_imputed)
    mean_temp = np.mean(frame_np_imputed)
    std_temp = np.std(frame_np_imputed)
    median_temp = np.median(frame_np_imputed)
    q1 = np.percentile(frame_np_imputed, 25)
    q3 = np.percentile(frame_np_imputed, 75)
    iqr = q3 - q1
    skewness = pd.DataFrame(frame_np_imputed.flatten()).skew()[0]
    kurtosis = pd.DataFrame(frame_np_imputed.flatten()).kurt()[0]
    high_temp_pixels = np.sum(frame_np_imputed > mean_temp + 1.5 * std_temp)

    # Append features
    feature_list.append([
        file_name, min_temp, max_temp, mean_temp, std_temp, median_temp, 
        q1, q3, iqr, skewness, kurtosis, high_temp_pixels
    ])

# Create DataFrame
feature_columns = ["Frame", "Min_Temp", "Max_Temp", "Mean_Temp", "Std_Temp", "Median_Temp", "Q1", "Q3", "IQR", "Skewness", "Kurtosis", "High_Temp_Pixels"]
df_features = pd.DataFrame(feature_list, columns=feature_columns)

# Debug: Show the first few extracted features
print(df_features.head())

# Save as CSV
df_features.to_csv("thermal_frame_features.csv", index=False)
print("✅ Debug Feature extraction completed. Check 'thermal_frame_features.csv'.")


Total Frames Found: 1564
            Frame  Min_Temp  Max_Temp    Mean_Temp    Std_Temp  Median_Temp  \
0     Frame_1.csv    1351.0    1875.0  1517.221680   69.055878       1510.0   
1    Frame_10.csv    1201.0    1719.0  1336.949829   90.495499       1301.0   
2   Frame_100.csv    1096.0    1690.0  1274.625977  101.334846       1238.0   
3  Frame_1000.csv    1034.0    1631.0  1185.342651   90.553833       1151.0   
4  Frame_1001.csv    1226.0    2107.0  1360.540894  116.824043       1330.0   

       Q1      Q3    IQR  Skewness   Kurtosis  High_Temp_Pixels  
0  1475.0  1526.0   51.0  2.292498   6.778652              2950  
1  1285.0  1330.0   45.0  2.086964   4.205517              3732  
2  1213.0  1315.0  102.0  1.584581   2.368822              3642  
3  1139.0  1217.0   78.0  1.923829   4.081294              3339  
4  1313.0  1360.0   47.0  3.668308  14.856552              2495  
✅ Debug Feature extraction completed. Check 'thermal_frame_features.csv'.
